# Exercise 9: LangGraph Multi-Agent System
## Simple Financial Planning with LangGraph State Management

In this notebook, we'll create a simple but powerful multi-agent financial planning system using LangGraph. We'll learn how to use state management, conditional routing, and agent coordination in a clean, graph-based approach.

## Learning Objectives
- Implement LangGraph for multi-agent workflows
- Use state management for agent coordination
- Create conditional routing between agents
- Build simple but effective financial planning workflows
- Understand graph-based agent orchestration

Let's build a streamlined financial planning system with LangGraph! 🔀📊🤖

## Setup and Imports

First, let's install and import the necessary libraries for our LangGraph multi-agent system.

In [3]:
# Install required packages
!uv add langchain langchain-openai langgraph python-dotenv

Resolved 392 packages in 1ms
Audited 223 packages in 0.07ms


In [4]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Optional
import os
from dotenv import load_dotenv
import json
import operator
from typing import Annotated

# Load environment variables
load_dotenv()

print("✓ Setup complete!")
print("✓ Ready to build LangGraph multi-agent system!")

✓ Setup complete!
✓ Ready to build LangGraph multi-agent system!


## Part 1: Define Agent State

LangGraph uses state management to coordinate between agents. Let's define our financial planning state structure.

In [5]:
# Define the state structure for our financial planning system
class FinancialPlanningState(TypedDict):
    """State structure for the financial planning workflow."""
    
    # Client information
    client_query: str
    client_profile: dict
    
    # Agent responses
    messages: Annotated[List[str], operator.add]
    recommendations: Annotated[List[str], operator.add]
    
    # Workflow control
    next_agent: Optional[str]
    completed_agents: Annotated[List[str], operator.add]
    final_recommendation: Optional[str]
    
    # Analysis results
    tax_analysis: Optional[str]
    investment_analysis: Optional[str]
    budget_analysis: Optional[str]

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3,
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ State structure defined!")
print("✅ LLM initialized!")

✅ State structure defined!
✅ LLM initialized!


## Part 2: Create Agent Functions

Now let's create simple but effective agent functions that work within the LangGraph framework.

In [6]:
# Tax Advisor Agent
def tax_advisor_agent(state: FinancialPlanningState):
    """Tax planning agent that analyzes tax implications."""
    
    client_query = state["client_query"]
    client_profile = state.get("client_profile", {})
    
    # Create tax analysis prompt
    tax_prompt = f"""
    You are a Tax Advisor. Analyze the client's situation for tax optimization opportunities.
    
    Client Query: {client_query}
    Client Profile: {json.dumps(client_profile, indent=2)}
    
    Provide:
    1. Tax situation analysis
    2. Optimization recommendations
    3. Whether investment or budget planning is needed (respond with NEED_INVESTMENT or NEED_BUDGET if so)
    
    Keep response concise and actionable.
    """
    
    response = llm.invoke([HumanMessage(content=tax_prompt)])
    analysis = response.content
    
    # Determine if other agents are needed
    next_agent = None
    if "NEED_INVESTMENT" in analysis:
        next_agent = "investment_analyst"
    elif "NEED_BUDGET" in analysis:
        next_agent = "budget_planner"
    
    return {
        "messages": [f"Tax Advisor: {analysis}"],
        "tax_analysis": analysis,
        "next_agent": next_agent,
        "completed_agents": ["tax_advisor"],
        "recommendations": ["Tax optimization strategies provided"]
    }

# Investment Analyst Agent  
def investment_analyst_agent(state: FinancialPlanningState):
    """Investment planning agent that provides portfolio recommendations."""
    
    client_query = state["client_query"]
    client_profile = state.get("client_profile", {})
    tax_analysis = state.get("tax_analysis", "")
    
    # Create investment analysis prompt
    investment_prompt = f"""
    You are an Investment Analyst. Analyze the client's investment needs.
    
    Client Query: {client_query}
    Client Profile: {json.dumps(client_profile, indent=2)}
    Previous Tax Analysis: {tax_analysis}
    
    Provide:
    1. Investment strategy recommendations
    2. Risk assessment
    3. Portfolio allocation suggestions
    4. Whether budget planning is needed (respond with NEED_BUDGET if so)
    
    Keep response concise and actionable.
    """
    
    response = llm.invoke([HumanMessage(content=investment_prompt)])
    analysis = response.content
    
    # Determine if budget planning is needed
    next_agent = "budget_planner" if "NEED_BUDGET" in analysis else None
    
    return {
        "messages": [f"Investment Analyst: {analysis}"],
        "investment_analysis": analysis,
        "next_agent": next_agent,
        "completed_agents": ["investment_analyst"],
        "recommendations": ["Investment strategy recommendations provided"]
    }

# Budget Planner Agent
def budget_planner_agent(state: FinancialPlanningState):
    """Budget planning agent that provides comprehensive financial planning."""
    
    client_query = state["client_query"]
    client_profile = state.get("client_profile", {})
    tax_analysis = state.get("tax_analysis", "")
    investment_analysis = state.get("investment_analysis", "")
    
    # Create budget analysis prompt
    budget_prompt = f"""
    You are a Budget Planner and Financial Coordinator. Provide comprehensive financial planning.
    
    Client Query: {client_query}
    Client Profile: {json.dumps(client_profile, indent=2)}
    Previous Tax Analysis: {tax_analysis}
    Previous Investment Analysis: {investment_analysis}
    
    Provide:
    1. Budget optimization recommendations
    2. Financial goal prioritization
    3. Integration of tax and investment strategies
    4. Action plan for the client
    
    Keep response concise and actionable.
    """
    
    response = llm.invoke([HumanMessage(content=budget_prompt)])
    analysis = response.content
    
    return {
        "messages": [f"Budget Planner: {analysis}"],
        "budget_analysis": analysis,
        "next_agent": None,  # Budget planner typically ends the workflow
        "completed_agents": ["budget_planner"],
        "recommendations": ["Comprehensive financial plan provided"]
    }

print("✅ Agent functions created!")
print("  🏛️ Tax Advisor - Tax optimization analysis")
print("  📈 Investment Analyst - Portfolio recommendations")
print("  💰 Budget Planner - Comprehensive financial planning")

✅ Agent functions created!
  🏛️ Tax Advisor - Tax optimization analysis
  📈 Investment Analyst - Portfolio recommendations
  💰 Budget Planner - Comprehensive financial planning


## Part 3: Router and Workflow Logic

Let's create the routing logic that determines which agent should run next based on the current state.

In [7]:
# Router function to determine next agent
def route_to_agent(state: FinancialPlanningState):
    """Route to the appropriate agent based on current state."""
    
    next_agent = state.get("next_agent")
    completed_agents = state.get("completed_agents", [])
    
    # If specific agent is requested, route there
    if next_agent:
        if next_agent == "investment_analyst" and "investment_analyst" not in completed_agents:
            return "investment_analyst"
        elif next_agent == "budget_planner" and "budget_planner" not in completed_agents:
            return "budget_planner"
    
    # Default routing based on query content
    client_query = state["client_query"].lower()
    
    # If no agents have run yet, start with appropriate agent
    if not completed_agents:
        if any(word in client_query for word in ["tax", "deduction", "irs"]):
            return "tax_advisor"
        elif any(word in client_query for word in ["invest", "portfolio", "stock"]):
            return "investment_analyst"
        else:
            return "budget_planner"
    
    # If we have analyses but no budget planning, go to budget planner
    if "budget_planner" not in completed_agents:
        return "budget_planner"
    
    # End the workflow
    return END

# Final summary function
def create_final_summary(state: FinancialPlanningState):
    """Create a final summary of all agent recommendations."""
    
    client_query = state["client_query"]
    messages = state.get("messages", [])
    recommendations = state.get("recommendations", [])
    
    summary_prompt = f"""
    Create a final summary for the client based on all agent analyses.
    
    Original Query: {client_query}
    
    Agent Analyses:
    {chr(10).join(messages)}
    
    Provide a concise, actionable summary with:
    1. Key findings from all agents
    2. Prioritized action items
    3. Next steps for the client
    
    Format as a clear, executive summary.
    """
    
    response = llm.invoke([HumanMessage(content=summary_prompt)])
    final_summary = response.content
    
    return {
        "final_recommendation": final_summary,
        "messages": [f"Final Summary: {final_summary}"]
    }

print("✅ Router and workflow logic created!")
print("  🔀 Smart routing based on query content and agent needs")
print("  📋 Final summary generation")
print("  🔄 State-based workflow control")

✅ Router and workflow logic created!
  🔀 Smart routing based on query content and agent needs
  📋 Final summary generation
  🔄 State-based workflow control


## Part 4: Build the LangGraph Workflow

Now let's build the actual LangGraph workflow that orchestrates our multi-agent system.

In [8]:
# Build the LangGraph workflow
def create_financial_planning_graph():
    """Create the LangGraph workflow for financial planning."""
    
    # Initialize the graph
    workflow = StateGraph(FinancialPlanningState)
    
    # Add agent nodes
    workflow.add_node("tax_advisor", tax_advisor_agent)
    workflow.add_node("investment_analyst", investment_analyst_agent)
    workflow.add_node("budget_planner", budget_planner_agent)
    workflow.add_node("final_summary", create_final_summary)
    
    # Add conditional routing
    workflow.add_conditional_edges(
        "tax_advisor",
        route_to_agent,
        {
            "investment_analyst": "investment_analyst",
            "budget_planner": "budget_planner",
            END: "final_summary"
        }
    )
    
    workflow.add_conditional_edges(
        "investment_analyst", 
        route_to_agent,
        {
            "budget_planner": "budget_planner",
            END: "final_summary"
        }
    )
    
    workflow.add_conditional_edges(
        "budget_planner",
        route_to_agent,
        {
            END: "final_summary"
        }
    )
    
    # Final summary always ends
    workflow.add_edge("final_summary", END)
    
    # Set entry point based on routing
    workflow.set_conditional_entry_point(
        route_to_agent,
        {
            "tax_advisor": "tax_advisor",
            "investment_analyst": "investment_analyst", 
            "budget_planner": "budget_planner"
        }
    )
    
    # Compile the graph
    return workflow.compile()

# Create the workflow
financial_planning_graph = create_financial_planning_graph()

print("✅ LangGraph workflow created!")
print("  🔀 Conditional routing between agents")
print("  📊 State management for coordination")
print("  🎯 Smart entry point selection")
print("  ✅ Compiled and ready to execute!")

✅ LangGraph workflow created!
  🔀 Conditional routing between agents
  📊 State management for coordination
  🎯 Smart entry point selection
  ✅ Compiled and ready to execute!


## Part 5: Simple Demo Functions

Let's create some simple demo functions to test our LangGraph multi-agent system.

In [9]:
# Simple demo function
def run_financial_planning(query: str, client_profile: dict = None):
    """Run the financial planning workflow with LangGraph."""
    
    print(f"🚀 STARTING FINANCIAL PLANNING WORKFLOW")
    print(f"Query: {query}")
    print("=" * 50)
    
    # Create initial state
    initial_state = {
        "client_query": query,
        "client_profile": client_profile or {},
        "messages": [],
        "recommendations": [],
        "completed_agents": [],
        "next_agent": None,
        "final_recommendation": None,
        "tax_analysis": None,
        "investment_analysis": None,
        "budget_analysis": None
    }
    
    # Run the workflow
    result = financial_planning_graph.invoke(initial_state)
    
    print("📋 WORKFLOW EXECUTION RESULTS:")
    print("=" * 50)
    
    # Show agent execution order
    completed_agents = result.get("completed_agents", [])
    print(f"🔄 Agents Executed: {' → '.join(completed_agents)}")
    
    # Show key analyses
    if result.get("tax_analysis"):
        print(f"\n🏛️ Tax Analysis: {result['tax_analysis'][:100]}...")
    
    if result.get("investment_analysis"):
        print(f"\n📈 Investment Analysis: {result['investment_analysis'][:100]}...")
    
    if result.get("budget_analysis"):
        print(f"\n💰 Budget Analysis: {result['budget_analysis'][:100]}...")
    
    # Show final recommendation
    if result.get("final_recommendation"):
        print(f"\n🎯 FINAL RECOMMENDATION:")
        print("=" * 30)
        print(result["final_recommendation"])
    
    print("\n✅ Workflow completed successfully!")
    return result

# Quick test function
def quick_test():
    """Quick test of the LangGraph system."""
    
    print("🧪 QUICK TEST OF LANGGRAPH SYSTEM")
    print("=" * 40)
    
    # Simple tax query
    result = run_financial_planning(
        "I make $80,000 per year and want to minimize my taxes while starting to invest.",
        {"age": 29, "annual_income": 80000, "current_savings": 15000}
    )
    
    return result

print("✅ Demo functions created!")
print("  🧪 Quick test function available")
print("  🚀 Full workflow execution function ready")
print("  📊 Results display and analysis included")

✅ Demo functions created!
  🧪 Quick test function available
  🚀 Full workflow execution function ready
  📊 Results display and analysis included


## Part 6: Run Demo

Let's test our LangGraph multi-agent system with a realistic financial planning scenario.

In [10]:
# Run the demo
print("🔥 RUNNING LANGGRAPH MULTI-AGENT DEMO")
print("=" * 60)

# Test with a comprehensive query
demo_result = run_financial_planning(
    "I'm 32 years old, make $95,000 per year, have $25,000 in savings, and want comprehensive financial planning including tax optimization and investment strategy.",
    {
        "age": 32,
        "annual_income": 95000,
        "monthly_income": 7917,
        "current_savings": 25000,
        "goals": ["tax_optimization", "investment_strategy", "retirement_planning"]
    }
)

print("\n" + "=" * 60)
print("🔍 WORKFLOW ANALYSIS")
print("=" * 60)

# Analyze the workflow execution
print(f"📊 Total Messages: {len(demo_result.get('messages', []))}")
print(f"🎯 Recommendations: {len(demo_result.get('recommendations', []))}")
print(f"🤖 Agents Used: {len(demo_result.get('completed_agents', []))}")

print("\n💡 Key Benefits of LangGraph Approach:")
print("  ✅ Simple, clean state management")
print("  ✅ Conditional routing between agents")
print("  ✅ Automatic workflow orchestration")
print("  ✅ Clear separation of concerns")
print("  ✅ Easy to extend and modify")

print("\n🎉 Demo completed successfully!")

🔥 RUNNING LANGGRAPH MULTI-AGENT DEMO
🚀 STARTING FINANCIAL PLANNING WORKFLOW
Query: I'm 32 years old, make $95,000 per year, have $25,000 in savings, and want comprehensive financial planning including tax optimization and investment strategy.
📋 WORKFLOW EXECUTION RESULTS:
🔄 Agents Executed: tax_advisor → investment_analyst → budget_planner

🏛️ Tax Analysis: 1. Tax Situation Analysis:
   - Your annual income of $95,000 puts you in a moderate tax bracket, po...

📈 Investment Analysis: 1. Investment strategy recommendations:
   - Consider a diversified investment portfolio that aligns...

💰 Budget Analysis: Budget Optimization Recommendations:
- Create a detailed budget to track expenses and allocate incom...

🎯 FINAL RECOMMENDATION:
Executive Summary:

Key Findings:
- Your income of $95,000 puts you in a moderate tax bracket, making you eligible for tax optimization strategies.
- With $25,000 in savings, you have the potential to invest in tax-efficient options.
- Your age and income lev

## Summary: LangGraph Multi-Agent Systems

Let's summarize what we've learned about building multi-agent systems with LangGraph.

In [11]:
# Summary of LangGraph concepts
print("📚 LANGGRAPH MULTI-AGENT CONCEPTS MASTERED")
print("=" * 60)

langgraph_concepts = {
    "State Management": {
        "description": "Centralized state shared across all agents",
        "benefit": "Clean data flow and coordination",
        "example": "FinancialPlanningState with typed fields"
    },
    "Conditional Routing": {
        "description": "Smart routing based on state and content",
        "benefit": "Dynamic workflow execution",
        "example": "route_to_agent() function with multiple conditions"
    },
    "Agent Nodes": {
        "description": "Individual agents as graph nodes",
        "benefit": "Modular, reusable components",
        "example": "tax_advisor_agent, investment_analyst_agent"
    },
    "Graph Compilation": {
        "description": "Workflow compiled into executable graph",
        "benefit": "Optimized execution and validation",
        "example": "workflow.compile() creates runnable graph"
    },
    "Automatic Orchestration": {
        "description": "LangGraph handles workflow execution",
        "benefit": "No manual coordination needed",
        "example": "financial_planning_graph.invoke(state)"
    }
}

for concept, details in langgraph_concepts.items():
    print(f"\n🔹 {concept}:")
    print(f"   What: {details['description']}")
    print(f"   Why: {details['benefit']}")
    print(f"   How: {details['example']}")

print("\n" + "=" * 60)
print("🏆 LANGGRAPH VS TRADITIONAL APPROACHES")
print("=" * 60)

print("Traditional Multi-Agent Systems:")
print("  ❌ Manual coordination logic")
print("  ❌ Complex state management")
print("  ❌ Hardcoded workflow paths")
print("  ❌ Difficult to debug and extend")

print("\nLangGraph Multi-Agent Systems:")
print("  ✅ Automatic orchestration")
print("  ✅ Clean state management")
print("  ✅ Conditional routing")
print("  ✅ Easy to visualize and debug")
print("  ✅ Highly extensible")

print("\n" + "=" * 60)
print("🎯 WHEN TO USE LANGGRAPH")
print("=" * 60)

use_cases = [
    "Multi-step workflows with conditional logic",
    "Complex agent coordination requirements", 
    "State-dependent routing between agents",
    "Workflows that need to be easily visualized",
    "Systems requiring robust error handling",
    "Applications with dynamic execution paths"
]

print("Perfect for:")
for i, use_case in enumerate(use_cases, 1):
    print(f"  {i}. {use_case}")

print("\n✅ Exercise 9 Complete: LangGraph Multi-Agent Systems Mastered!")
print("🚀 You now know how to build clean, efficient multi-agent systems!")

📚 LANGGRAPH MULTI-AGENT CONCEPTS MASTERED

🔹 State Management:
   What: Centralized state shared across all agents
   Why: Clean data flow and coordination
   How: FinancialPlanningState with typed fields

🔹 Conditional Routing:
   What: Smart routing based on state and content
   Why: Dynamic workflow execution
   How: route_to_agent() function with multiple conditions

🔹 Agent Nodes:
   What: Individual agents as graph nodes
   Why: Modular, reusable components
   How: tax_advisor_agent, investment_analyst_agent

🔹 Graph Compilation:
   What: Workflow compiled into executable graph
   Why: Optimized execution and validation
   How: workflow.compile() creates runnable graph

🔹 Automatic Orchestration:
   What: LangGraph handles workflow execution
   Why: No manual coordination needed
   How: financial_planning_graph.invoke(state)

🏆 LANGGRAPH VS TRADITIONAL APPROACHES
Traditional Multi-Agent Systems:
  ❌ Manual coordination logic
  ❌ Complex state management
  ❌ Hardcoded workflow pat